In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv


In [2]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import io
import json
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from tensorflow.keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing import text, sequence
from gensim.models import KeyedVectors
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential

import transformers
from tokenizers import BertWordPieceTokenizer

In [3]:
PATH_TPU_WORKER = ''

def check_tpu():
    """
    Detect TPU hardware and return the appopriate distribution strategy
    """
    
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
        print('Running on TPU: {}'.format(tpu.master()))
    except ValueError:
        tpu = None
    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        tpu_strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.
        print("Num. replicas: {}".format(tpu_strategy.num_replicas_in_sync))
    
    return tpu, tpu_strategy
    
tpu, tpu_strategy = check_tpu()
PATH_TPU_WORKER = tpu.master()
NUM_REPLICAS = tpu_strategy.num_replicas_in_sync

Running on TPU: grpc://10.0.0.2:8470


In [4]:
PATH_CHALLENGE = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'

PATH_TRAIN_FILENAME = PATH_CHALLENGE + "jigsaw-toxic-comment-train.csv"
PATH_TEST_FILENAME = PATH_CHALLENGE + "test.csv"
PATH_VALID_FILENAME = PATH_CHALLENGE + "validation.csv"


"""
LOAD
"""

train_df = pd.read_csv(PATH_TRAIN_FILENAME)
test_df = pd.read_csv(PATH_TEST_FILENAME)
valid_df = pd.read_csv(PATH_VALID_FILENAME)

In [5]:
MAX_LEN = 256

# Adapted https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

save_path = '/kaggle/working/distilbert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)

fast_tokenizer = BertWordPieceTokenizer('distilbert_base_uncased/vocab.txt', lowercase=True)


In [6]:
def encode(texts, tokenizer, chunk_size=256, maxlen=MAX_LEN):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(max_length=maxlen)
    all_ids = []
    
    for i in range(0, len(texts), chunk_size):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

x_train = encode(train_df.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = encode(valid_df.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = encode(test_df.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)
y_train = train_df.toxic.values
y_valid = valid_df.toxic.values


In [7]:
AUTO = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 16 
TOTAL_BATCH_SIZE = BATCH_SIZE * tpu_strategy.num_replicas_in_sync
print("Batch size: {}".format(BATCH_SIZE))
print("Total batch size: {}".format(TOTAL_BATCH_SIZE))

Batch size: 16
Total batch size: 128


In [8]:
"""
DATA Loading
"""


train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(TOTAL_BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

valid_dataset = (tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    #.repeat()
    .batch(TOTAL_BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(TOTAL_BATCH_SIZE)
    #.repeat()
    .cache()
    .prefetch(AUTO)
)


In [9]:
def simple_model(max_len=MAX_LEN):
    words = Input(shape=(max_len,), batch_size=TOTAL_BATCH_SIZE, dtype=tf.int32, name="words")
    x = Dense(10, activation='relu')(words)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=words, outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model.summary()

simple_model()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(128, 256)]              0         
_________________________________________________________________
dense (Dense)                (128, 10)                 2570      
_________________________________________________________________
dense_1 (Dense)              (128, 1)                  11        
Total params: 2,581
Trainable params: 2,581
Non-trainable params: 0
_________________________________________________________________


In [10]:
with tpu_strategy.scope():
    simple_model()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
words (InputLayer)           [(16, 256)]               0         
_________________________________________________________________
dense_2 (Dense)              (16, 10)                  2570      
_________________________________________________________________
dense_3 (Dense)              (16, 1)                   11        
Total params: 2,581
Trainable params: 2,581
Non-trainable params: 0
_________________________________________________________________


In [11]:
def lstm_model(vocab_size, max_len=MAX_LEN):
    
    words = Input(shape=(max_len,), dtype=tf.int32, name="words")
    x = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=512, input_length=max_len)(words)
    x = tf.keras.layers.SpatialDropout1D(0.3)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)

    hidden = tf.keras.layers.concatenate([
        tf.keras.layers.GlobalMaxPooling1D()(x),
        tf.keras.layers.GlobalAveragePooling1D()(x),
    ])
    hidden = tf.keras.layers.add([hidden, Dense(4 * 256, activation='relu')(hidden)])
    hidden = tf.keras.layers.add([hidden, Dense(4 * 256, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    model = tf.keras.Model(inputs=words, outputs=result)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), # much more faster with 
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=['accuracy'])
    return model
"""
BUILD
"""

with tpu_strategy.scope():
    vocab_size = tokenizer.vocab_size # Distil
    model = lstm_model(vocab_size)
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words (InputLayer)              [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 256, 512)     61208064    words[0][0]                      
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 256, 512)     0           embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 256, 512)     1574912     spatial_dropout1d[0][0]          
____________________________________________________________________________________________

In [12]:
EPOCHS = 5

N_TRAIN_STEPS = 219
N_VALID_STEPS = 63
train_history = model.fit(
    train_dataset,
    steps_per_epoch=N_TRAIN_STEPS,
    validation_data=valid_dataset,
    validation_steps=N_VALID_STEPS,
    epochs=EPOCHS
)


def auc_roc(dataset, ground_truth):
    from sklearn.metrics import roc_curve
    y_pred_keras = model.predict(dataset, verbose=1).ravel()
    fpr_keras, tpr_keras, thresholds_keras = roc_curve(ground_truth, y_pred_keras)
    from sklearn.metrics import auc
    return auc(fpr_keras, tpr_keras)

print("AUC-ROC validation set: ")
auc_roc(valid_dataset, y_valid)

Train for 219 steps, validate for 63 steps
Epoch 1/5
219/219 [==============================] - 57s 259ms/step - loss: 0.2602 - accuracy: 0.9144 - val_loss: 0.6932 - val_accuracy: 0.5955
Epoch 2/5
219/219 [==============================] - 41s 187ms/step - loss: 0.1469 - accuracy: 0.9473 - val_loss: 0.5992 - val_accuracy: 0.7226
Epoch 3/5
219/219 [==============================] - 41s 188ms/step - loss: 0.1261 - accuracy: 0.9554 - val_loss: 0.4725 - val_accuracy: 0.8279
Epoch 4/5
219/219 [==============================] - 41s 187ms/step - loss: 0.1133 - accuracy: 0.9591 - val_loss: 0.4934 - val_accuracy: 0.8346
Epoch 5/5
219/219 [==============================] - 41s 187ms/step - loss: 0.1115 - accuracy: 0.9590 - val_loss: 0.5337 - val_accuracy: 0.8341
AUC-ROC validation set: 
63/63 [==============================] - 5s 79ms/step


0.6195944566535768

In [13]:
predictions = model.predict(test_dataset, verbose=1).ravel()

input_size = test_df.shape[0]
output_size = predictions.shape[0]

if input_size != output_size:
    print("Input size differs from output size. Input size: {}, Output size: {}".format(input_size,output_size))

if output_size % NUM_REPLICAS == 0:
    print("Predicitions is divisible by ".format(NUM_REPLICAS))
    
    
submission = pd.DataFrame.from_dict({
    'id': test_df.id,
    'toxic': predictions[:input_size]
})

print("Save submission to csv.")
submission.to_csv('submission.csv', index=False)

499/499 [==============================] - 21s 41ms/step
Input size differs from output size. Input size: 63812, Output size: 63816
Predicitions is divisible by 
Save submission to csv.
